<a href="https://colab.research.google.com/github/ezemaudet/TP_NLP1/blob/main/Desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [22]:
tfidfvect = TfidfVectorizer(
    strip_accents='unicode',
    lowercase=True,
    stop_words='english',
    ngram_range=(1, 2),
    max_df=0.85,
    min_df=1,
    max_features=None,
    sublinear_tf=True,
    norm='l2'
)

In [23]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
import random

# Elegir 5 índices al azar
indices = random.sample(range(X_train.shape[0]), 5)

# Extraer solo esos 5 documentos de X_train
subset = X_train[indices]

# Calcular matriz de similitud coseno entre los 5 documentos
sim_matrix = cosine_similarity(subset)

# Mostrar resultados
import pandas as pd
df = pd.DataFrame(sim_matrix, index=indices, columns=indices)
print(df.round(3))


       4622   9861   8993   8096   595 
4622  1.000  0.000  0.040  0.007  0.000
9861  0.000  1.000  0.001  0.000  0.002
8993  0.040  0.001  1.000  0.002  0.003
8096  0.007  0.000  0.002  1.000  0.000
595   0.000  0.002  0.003  0.000  1.000


In [25]:
# y los 5 más similares son de las clases:
for i in indices:
  print(newsgroups_train.target_names[y_train[i]])


alt.atheism
sci.space
talk.religion.misc
rec.sport.hockey
alt.atheism


In [26]:
# y los 5 más similares son de las clases:
for i in range(5):
  print(newsgroups_train.data[indices[i]])
  print("---------------------ooooooo-----------------------")



Who has to consider it?  The being that does the action?  I'm still
not sure I know what you are trying to say.
---------------------ooooooo-----------------------
In reference to the limits of acceleration with guns launching solid
rockets as payloads.  Thiokol provided me with samples and data on
a reinforcement to solid motor grains for high accelerations.  Solid
motor propellants usually have a substantial percentage of 
aluminum in the mix.  For example, the Space Shuttle SRBs are 16 percent
Aluminum.  The technique is to use a 'foamed aluminum' structure.
The structure looks like the inverse of a set of bubbles (an I suspect
some bubbling process is used to form it).  In other words, if you made
a bunch of bubbles in molten aluminum, then froze it, this is what
you get.  It forms a strong network of effectively aluminum wires in
all directions.  The remaining solid fuel mix is infiltrated into
the voids, and you get aluminum-reinforced solid propellant.  The
foamed-aluminum mak

In [27]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [28]:
mostsim

array([6635, 4253, 3596, 4271, 3746])

In [29]:
# y los 5 más similares son de las clases:
for i in range(5):
  print(newsgroups_train.data[mostsim[i]])


THE WHITE HOUSE

                    Office of the Press Secretary
______________________________________________________________
For Immediate Release                             April 15, 1993     

	     
                       REMARKS BY THE PRESIDENT
                   TO LAW ENFORCEMENT ORGANIZATIONS
	     
	     
                           The Rose Garden 


2:52 P.M. EDT


	     THE PRESIDENT:  Good afternoon.  Ladies and gentlemen, 
two months ago I presented a comprehensive plan to reduce our 
national deficit and to increase our investment in the American 
people, their jobs and their economic future.  The federal budget 
plan passed Congress in record time, and created a new sense of hope 
and opportunity in the country.  
	     
	     Then, the short-term jobs plan I presented to Congress, 
which would create a half a million jobs in the next two years passed 
the House of Representatives two weeks ago.  It now has the support 
of a majority of the United States Senate. 
	

2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [30]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB( alpha=1e-3, force_alpha=True, fit_prior=False, class_prior=None)
clf.fit(X_train, y_train)

MultinomialNB(alpha=0.001, fit_prior=False)

In [31]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [32]:
f1_score(y_test, y_pred, average='macro')

0.6860751293275787

Con Vectorizador por default f1: 0.6765487308188942
Con Vectorizador instanciado diferente f1; 0.6860
El cambio mas relevante fue llevar el ngram_range=(1, 2),

#**NB Complemento**

In [33]:
clf = ComplementNB( alpha=.1, force_alpha=True, fit_prior=False, class_prior=None, norm=False)
clf.fit(X_train, y_train)

ComplementNB(alpha=0.1, fit_prior=False)

In [34]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [35]:
f1_score(y_test, y_pred, average='macro')

0.7078395677748388

Con Vectorizador por default f1: 0.6953652590540836
Con Vectorizador instanciado diferente f1; 0.7078395677748388
El cambio mas relevante fue llevar el ngram_range=(1, 2),

3. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [36]:
X_train_T = X_train.T

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
import random

# Elegir 5 índices al azar
indices = random.sample(range(X_train_T.shape[0]), 5)

# Extraer solo esos 5 documentos de X_train
subset = X_train_T[indices]

# Calcular matriz de similitud coseno entre los 5 documentos
sim_matrix = cosine_similarity(subset)

# Mostrar resultados
import pandas as pd
df = pd.DataFrame(sim_matrix, index=indices, columns=indices)
print(df.round(3))

        119805  182859  403653  936724  800734
119805     1.0     0.0     0.0     0.0     0.0
182859     0.0     1.0     0.0     0.0     0.0
403653     0.0     0.0     1.0     0.0     0.0
936724     0.0     0.0     0.0     1.0     0.0
800734     0.0     0.0     0.0     0.0     1.0


In [38]:
X_train_T.shape

(943736, 11314)

In [39]:
X_train_T.data[27824]

np.float64(0.010157091692774936)

In [40]:
X_train.shape

(11314, 943736)

In [41]:
X_train.data[27824]

np.float64(0.010157091692774936)

Los IDX no cambian al transponer la matriz de Training.

In [42]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train_T[idx], X_train_T)[0]

In [43]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1., 1., 1., ..., 0., 0., 0.])

In [44]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([486626, 336093, 336095, ..., 627701, 627700, 627714])

In [45]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:50]

In [46]:
mostsim

array([336093, 336095, 774383,  50358,  50359, 774378, 486606, 336115,
       336121, 336122, 682038, 461155, 461154, 461153, 525171, 336135,
       883497, 336136, 367631, 303263, 336137, 883502, 461156, 344786,
       603895, 461196, 461195, 461194, 461193,  78050, 303239, 344782,
        78044, 461188, 883478, 571622, 571623, 344779, 571628, 571629,
       774390, 486627, 883477, 461178, 461177, 461175, 461174, 461187,
       303278])

In [47]:
feature_names = tfidfvect.get_feature_names_out()
# índice de la fila en la matriz transpuesta
print(f"Palabra #{378409}: {feature_names[378409]}")

Palabra #378409: gets free


In [48]:
tfidfvect.vocabulary_['gets free']

378409

In [49]:
tfidfvect.vocabulary_['senior administration']

756491

In [50]:
tfidfvect.vocabulary_['senior']

756489

In [51]:
tfidfvect.vocabulary_['administration']

95971

Aca se ve que los vectores 756491 y 756489 estan proximos y se comoponen de palabras similares

In [52]:
cosine_similarity(X_train_T[756489], X_train_T)[0]

array([0.00348053, 0.00037069, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [53]:
feature_names = tfidfvect.get_feature_names_out()
# índice de la fila en la matriz transpuesta
print(f"Palabra #{486626}: {feature_names[303278]}")

Palabra #486626: ea808zimp


In [54]:
idx1 = list(feature_names).index("senior")
idx2 = list(feature_names).index("administration")

from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(X_train_T[idx1], X_train_T[idx2])[0][0]
print(f"Similitud coseno : {sim:.4f}")


Similitud coseno : 0.0849


In [55]:
idx1 = list(feature_names).index("president")
idx2 = list(feature_names).index("government")

from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(X_train_T[idx1], X_train_T[idx2])[0][0]
print(f"Similitud coseno : {sim:.4f}")


Similitud coseno : 0.1057


No veo mucha similitud en las palabras. Como ejemplo ustilice president y governmet que deberian estar altamente relacionadas pero la similitud coseno solo es de 0.1
Dos palabras tendrán vectores similares si aparecen en los mismos documentos con pesos similares.

In [56]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:200]

Codigo para extraer palabras con similitud de coseno en ciertos intervalos

In [57]:

base_idx = 1000
base_vector = X_train_T[base_idx]
cossim = cosine_similarity(base_vector, X_train_T)[0]

# Obtenemos los nombres de las palabras
feature_names = tfidfvect.get_feature_names_out()

# Definimos los niveles de similitud que queremos estudiar
niveles = [1.0, 0.9, 0.8, 0.7, 0.6]

# Para cada nivel, extraer hasta 10 palabras cuya similitud esté en ese rango
for nivel in niveles:
    print(f"\n🔹 Palabras con similitud cercana a {nivel}:")
    palabras_en_rango = [
        i for i in np.argsort(cossim)[::-1]  # ordenamos por similitud descendente
        if i != base_idx and nivel - 0.05 <= cossim[i] <= nivel + 0.05
    ]
    for i in palabras_en_rango[:10]:  # tomamos hasta 10
        print(f"{feature_names[i]} (sim: {cossim[i]:.3f})")



🔹 Palabras con similitud cercana a 1.0:
sniffer files (sim: 1.000)
sniffer file (sim: 1.000)
better following (sim: 1.000)
sniffer (sim: 1.000)
sun ultrix (sim: 1.000)
number frames (sim: 1.000)
needed reading (sim: 1.000)
sun tar (sim: 1.000)
networks distributed (sim: 1.000)
tttt prints (sim: 1.000)

🔹 Palabras con similitud cercana a 0.9:
added allow (sim: 0.933)
00 15 (sim: 0.930)
processing data (sim: 0.914)
36 23 (sim: 0.908)
0020 (sim: 0.903)
stdout (sim: 0.856)
motif interface (sim: 0.852)

🔹 Palabras con similitud cercana a 0.8:
time number (sim: 0.849)
5b (sim: 0.844)
easily follow (sim: 0.819)
7c (sim: 0.810)
_p (sim: 0.806)
ftp win (sim: 0.790)
1285 (sim: 0.763)

🔹 Palabras con similitud cercana a 0.7:
16 36 (sim: 0.722)
5f (sim: 0.713)
reads writes (sim: 0.706)
good tools (sim: 0.701)
gcc dec (sim: 0.700)
difficult use (sim: 0.675)

🔹 Palabras con similitud cercana a 0.6:
link level (sim: 0.588)
sample output (sim: 0.567)
16384 (sim: 0.563)
result problems (sim: 0.562)


De esta lista rescato las sigueintes palabras con similitud
processing data (sim: 0.914)
result problems (sim: 0.562)


In [58]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Palabras a las que queremos encontrar otras similares
palabras_base = ['president', 'government', 'patient', 'science', 'game']

# Obtener vocabulario e índice inverso
vocabulario = tfidfvect.vocabulary_
feature_names = tfidfvect.get_feature_names_out()

print("\n🔍 Palabras más similares por similitud coseno:")
for palabra in palabras_base:
    if palabra not in vocabulario:
        print(f"⚠️ La palabra '{palabra}' no está en el vocabulario.")
        continue

    base_idx = vocabulario[palabra]
    base_vector = X_train_T[base_idx]
    cossim = cosine_similarity(base_vector, X_train_T)[0]

    # Obtener los índices de las palabras más similares (excluyendo a sí misma)
    palabras_similares = [
        i for i in np.argsort(cossim)[::-1] if i != base_idx
    ][:10]

    print(f"\n🔹 Palabra base: '{palabra}'")
    for i in palabras_similares:
        print(f"   {feature_names[i]} (sim: {cossim[i]:.3f})")



🔍 Palabras más similares por similitud coseno:

🔹 Palabra base: 'president'
   office press (sim: 0.403)
   immediate release (sim: 0.382)
   president easily (sim: 0.378)
   house office (sim: 0.376)
   press secretary (sim: 0.375)
   president clinton (sim: 0.373)
   easily wrong (sim: 0.359)
   vice president (sim: 0.353)
   release april (sim: 0.341)
   clinton president (sim: 0.308)

🔹 Palabra base: 'government'
   government agencies (sim: 0.228)
   federal government (sim: 0.213)
   encryption (sim: 0.200)
   agencies (sim: 0.197)
   limited government (sim: 0.196)
   people (sim: 0.193)
   clipper (sim: 0.192)
   citizens (sim: 0.192)
   libertarian (sim: 0.191)
   secure (sim: 0.186)

🔹 Palabra base: 'patient'
   patient feels (sim: 0.305)
   patient doesn (sim: 0.298)
   chest prescribe (sim: 0.294)
   feel cheated (sim: 0.294)
   cases physicians (sim: 0.294)
   demand cases (sim: 0.294)
   best taking (sim: 0.294)
   elastic binder (sim: 0.294)
   breath patient (sim: 0.29

Aca se observan que las palabras de la matrix transpuesta guardan similitud.

🔹 Palabra base: 'president'
   office press (sim: 0.403)
   immediate release (sim: 0.382)
   president easily (sim: 0.378)
   house office (sim: 0.376)
   press secretary (sim: 0.375)
   president clinton (sim: 0.373)
   easily wrong (sim: 0.359)
   vice president (sim: 0.353)
   release april (sim: 0.341)
   clinton president (sim: 0.308)

   🔹 Palabra base: 'game'
   game bat (sim: 0.236)
   games (sim: 0.220)
   hockey (sim: 0.207)
   team (sim: 0.201)
   game game (sim: 0.194)
   playoffs (sim: 0.191)
   wings game (sim: 0.191)
   baseball game (sim: 0.190)
   espn (sim: 0.188)
   detroit (sim: 0.185)

KMeans

In [60]:
from sklearn.cluster import KMeans

k = 20  # número de clusters
kmeans = KMeans(n_clusters=k, random_state=42)
word_clusters = kmeans.fit_predict(X_train_T)


Agglomerative Clustering (jerárquico) esta clusterizacion no me corre polos recursos de colab

Asociar palabras a sus clusters

In [61]:
#feature_names = tfidfvect.get_feature_names_out()
clusters = {}
for idx, cluster_id in enumerate(word_clusters):
    clusters.setdefault(cluster_id, []).append(idx)


Medir similitud coseno dentro de un cluster

In [62]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [63]:


# Ejemplo: medir similitud entre palabras del cluster 15
indices = clusters[15]
subset = X_train_T[indices]
sim_matrix = cosine_similarity(subset)

# Visualizar las palabras más similares dentro del cluster
import pandas as pd
palabras = [feature_names[i] for i in indices]
df = pd.DataFrame(sim_matrix, index=palabras, columns=palabras)
print(df.round(2))


                    apple  apple dealer  appropriate  appropriate noises  \
apple                1.00          0.27         0.02                0.05   
apple dealer         0.27          1.00         0.03                0.32   
appropriate          0.02          0.03         1.00                0.11   
appropriate noises   0.05          0.32         0.11                1.00   
btw duo              0.05          0.32         0.11                1.00   
...                   ...           ...          ...                 ...   
vram floppy          0.05          0.32         0.11                1.00   
want slide           0.05          0.32         0.11                1.00   
way nowadays         0.05          0.32         0.11                1.00   
weeks                0.04          0.02         0.02                0.07   
weeks make           0.03          0.20         0.07                0.64   

                    btw duo  colour  colour display  come despite  come dock  \
apple  

Conclusion: Las palabras tendrán vectores similares si aparecen en los mismos documentos con pesos similares. Se clusterizo los vectores de palabras de la matriz transpuesta y se observa que guardan similitud.